In [ ]:
#cmd에서 root폴더 위치에 설치하기

#pip install ultralytics pyyaml pandas matplotlib seaborn opencv-python

# root 폴더 : ObjectDetection_SelectStar_Football

# 데이터셋 링크 다운로드 출처 : https://open.selectstar.ai/ko/fitogether
# fittogether.zip 을 압축을 풀어서 Data 폴더에 넣기
# Data\\fittogether 안에 *.jpg, *.json 파일들이 각각 11150개씩 들어있음을 확인하자.

In [33]:
import torch
import os
import json
import yaml
import shutil
import random
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from tqdm.auto import tqdm  # 로컬PC 환경에 최적화된 tqdm 로드

# ---------------------------------------------------------
# [단계 1] GPU 가속 환경 확인
# ---------------------------------------------------------
# 로컬 PC의 NVIDIA RTX GPU(CUDA)가 PyTorch에서 인식되는지 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f">> 장치 확인: {device}")
if torch.cuda.is_available():
    print(f">> GPU 모델: {torch.cuda.get_device_name(0)}")

>> 장치 확인: cuda
>> GPU 모델: NVIDIA GeForce RTX 4070 Laptop GPU


In [ ]:
# ---------------------------------------------------------
# [단계 2] 경로 설정 및 실험 변수 정의
# ---------------------------------------------------------
ROOT_DIR = "C:\\yssong\\ObjectDetection_SelectStar_Football"           # 프로젝트 최상위 경로
DATA_DIR = ROOT_DIR + "\\Data\\fittogether\\"  # 원본 JPG/JSON 데이터 경로

# 데이터 샘플링 비율 (0.1 = 전체의 10%만 사용하여 빠른 실험 진행)

DATA_RATIO = 0.1  # 테스트용 10% 사용
#DATA_RATIO = 1.0  # 전체 데이터 사용

# 학습 횟수 고정 (테스트용 1회, 실제 학습 시 30회 이상 권장)

#FIXED_EPOCHS = 5
FIXED_EPOCHS = 50

In [35]:
# ---------------------------------------------------------
# [단계 3] 클래스(객체 종류) 정의
# ---------------------------------------------------------
# YOLO 모델이 구분할 객체 이름 (B:공, Ta:팀A, Tb:팀B, R:심판, O:기타)
class_names = ['B', 'Ta', 'Tb', 'R', 'O']

# ---------------------------------------------------------
# [단계 4] 데이터 전처리 및 YOLO 포맷 변환 함수
# ---------------------------------------------------------
def prepare_final_data(files, split_name):
    # 각 분할(train/valid/test)에 맞는 이미지 및 라벨 폴더 생성
    img_dest = ROOT_DIR + f"\\{split_name}\\images"
    lbl_dest = ROOT_DIR + f"\\{split_name}\\labels"
    os.makedirs(img_dest, exist_ok=True); os.makedirs(lbl_dest, exist_ok=True)
    
    for img_path in tqdm(files, desc=f">> {split_name} 데이터 변환 중"):
        # 1. 이미지 파일을 대상 폴더로 복사
        shutil.copy(img_path, img_dest + "\\" + os.path.basename(img_path))
        
        # 2. 원본 JSON 어노테이션 파일 읽기
        json_path = img_path.replace('.jpg', '.json')

        if os.path.exists(json_path):
            img = cv2.imread(img_path); h, w, _ = img.shape
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            yolo_lines = []
            for shape in data['shapes']:
                label = shape['label']
                # [로직] 사용자 정의 클래스 번호(cid) 할당
                if label == 'ball': cid = 0
                elif label == 'players':
                    # JSON의 id 속성을 활용해 임시로 팀A(짝수)/팀B(홀수)를 구분
                    cid = 1 if shape.get('id', 0) % 2 == 0 else 2
                elif label == 'referee': cid = 3
                elif label == 'others': cid = 4
                else: continue
                
                # 3. 좌표 정규화 (YOLO 필수 포맷)
                # 원본 [x1, y1, x2, y2]를 이미지 크기(w, h) 대비 0~1 사이의 [상대중심x, 상대중심y, 상대너비, 상대높이]로 변환
                (x1, y1), (x2, y2) = shape['points']
                cx = (x1 + x2) / 2 / w       # 객체 중심 X좌표
                cy = (y1 + y2) / 2 / h       # 객체 중심 Y좌표
                bw = abs(x2 - x1) / w        # 객체 너비
                bh = abs(y2 - y1) / h        # 객체 높이
                yolo_lines.append(f"{cid} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")
            
            # 4. 변환된 정보를 .txt 파일로 저장 (YOLO 학습용 라벨 파일)
            with open(lbl_dest + "\\" + os.path.basename(img_path).replace('.jpg', '.txt'), 'w') as f:
                f.write('\n'.join(yolo_lines))

In [36]:
# ---------------------------------------------------------
# [단계 5] 데이터 샘플링 및 세트 분할 (7:1:2 비율)
# ---------------------------------------------------------
all_jpgs = glob(DATA_DIR + "*.jpg")
# DATA_RATIO에 따라 전체 중 일부만 무작위 추출
sampled_jpgs = random.sample(all_jpgs, int(len(all_jpgs) * DATA_RATIO))

# 전체 데이터를 Train(학습), Test(최종 평가)로 8:2 분리
train_f, test_f = train_test_split(sampled_jpgs, test_size=0.2, random_state=42)
# 학습 데이터를 다시 Train(학습), Valid(검증)로 7:1 비율이 되도록 재분리
train_f, val_f = train_test_split(train_f, test_size=0.125, random_state=42)

# 전처리 함수 실행 (파일 이동 및 변환)
for s_name, f_list in [('train', train_f), ('valid', val_f), ('test', test_f)]:
    prepare_final_data(f_list, s_name)

# ---------------------------------------------------------
# [단계 6] YOLO 데이터 설정 파일(YAML) 생성
# ---------------------------------------------------------
# 학습 시 모델이 데이터 경로와 클래스 정보를 알 수 있도록 설정 파일을 자동 생성합니다.
with open(ROOT_DIR + "\\fittogether.yaml", 'w') as f:
    yaml.dump({'train': ROOT_DIR+"\\train\\images", 
               'val': ROOT_DIR+"\\valid\\images", 
               'test': ROOT_DIR+"\\test\\images", 
               'nc': 5, 'names': class_names}, f)

>> train 데이터 변환 중:   0%|          | 0/780 [00:00<?, ?it/s]

>> valid 데이터 변환 중:   0%|          | 0/112 [00:00<?, ?it/s]

>> test 데이터 변환 중:   0%|          | 0/223 [00:00<?, ?it/s]

In [37]:
# ---------------------------------------------------------
# [단계 6-1] Step 1: 640 해상도에서 모델 성능 비교 (예비 실험)
# ---------------------------------------------------------
# 이 셀을 실행하면 640 해상도로 두 모델을 비교하고 결과를 저장합니다.
# PC를 끄고 다시 켜도 결과가 보존됩니다.

# json은 셀 2에서 이미 import했으므로 여기서는 불필요

# [설정] 초기 변수
models_to_compare = ['yolov8n.pt', 'yolov8s.pt']

base_sz = 640 # 비교 실험 해상도

# 결과를 저장할 딕셔너리
comparison_results = {}

print("="*70)
print(">> [Step 1] 640 해상도에서 모델 성능 비교 실험 시작")
print("="*70)

# 640 해상도에서 각 모델 학습
for m_pt in tqdm(models_to_compare, desc=">> Step 1 모델 비교"):
    m_tag = m_pt.split('.')[0]
    run_name = f"Step1_{m_tag}_SZ{base_sz}_E{FIXED_EPOCHS}"
    
    print(f"\n>> [{m_tag}] 모델 640 학습 시작...")
    
    try:
        model = YOLO(m_pt)
        model.train(
            data = ROOT_DIR + "\\fittogether.yaml",
            epochs = FIXED_EPOCHS,
            imgsz = base_sz,
            batch = -1,
            patience = 10,      # [핵심] 10회 연속 개선 없으면 조기 종료
            device = 0,
            project = ROOT_DIR + "\\runs",
            name = run_name,
            exist_ok = True,
            verbose = False
        )
        
        # 학습 완료 후 results.csv에서 최종 mAP50 읽기
        csv_path = f"{ROOT_DIR}\\runs\\{run_name}\\results.csv"
        if os.path.exists(csv_path):
            df_result = pd.read_csv(csv_path)
            df_result.columns = df_result.columns.str.strip()
            final_map = df_result['metrics/mAP50(B)'].iloc[-1]
            comparison_results[m_tag] = float(final_map)
            print(f">> {m_tag} 최종 mAP50: {final_map:.4f}")
        else:
            print(f"!! {m_tag} 결과 파일을 찾을 수 없습니다: {csv_path}")
            
    except Exception as e:
        print(f"!! {m_tag} 학습 중 오류 발생: {str(e)}")
        continue

# 결과 확인
if not comparison_results:
    print("\n!! 학습된 모델 결과가 없습니다. 위의 오류를 확인하세요.")
else:
    # 결과를 JSON 파일로 저장 (PC 재시작 후에도 사용 가능)
    results_file = ROOT_DIR + "\\step1_comparison_results.json"
    with open(results_file, 'w') as f:
        json.dump(comparison_results, f, indent=4)

    print("\n" + "="*70)
    print(f">> Step 1 완료! 결과가 저장되었습니다: {results_file}")
    print("="*70)
    print("\n[비교 결과]")
    for model, score in comparison_results.items():
        print(f"  {model}: mAP50 = {score:.4f}")

    winner_tag = max(comparison_results, key=comparison_results.get)
    print(f"\n>> 승자 모델: {winner_tag} (mAP50: {comparison_results[winner_tag]:.4f})")
    print(f"\n>> 다음 단계: 아래 Step 2 셀을 실행하여 {winner_tag} 모델로 1280 해상도 실험을 진행하세요.")


>> [Step 1] 640 해상도에서 모델 성능 비교 실험 시작


>> Step 1 모델 비교:   0%|          | 0/2 [00:00<?, ?it/s]


>> [yolov8n] 모델 640 학습 시작...
Ultralytics 8.3.249  Python-3.12.10 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\yssong\SelectStar\fittogether.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=Step1_yolov8n_SZ640_E50, nbs=64, nms=False, opset=None, optimize=False, optimiz

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000002517FEEA8E0>
Traceback (most recent call last):
  File "c:\Users\sjowu\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "c:\Users\sjowu\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py", line 1435, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


     3011823       8.198         1.623         39.05         57.45        (1, 3, 640, 640)                    list
     3011823        16.4         1.644         57.68         107.2        (2, 3, 640, 640)                    list
     3011823       32.79         1.753         15.73         24.59        (4, 3, 640, 640)                    list
     3011823       65.59         2.319         21.63         31.57        (8, 3, 640, 640)                    list
     3011823       131.2         3.328         39.23         58.28       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 13 for CUDA:0 6.52G/8.00G (82%) 
train: Fast image access  (ping: 0.00.0 ms, read: 1856.2459.1 MB/s, size: 361.8 KB)
train: Scanning C:\yssong\SelectStar\train\labels.cache... 780 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 780/780  0.0s
val: Fast image access  (ping: 0.00.0 ms, read: 930.9164.8 MB/s, size: 312.1 KB)
val: Scanning C:\yssong\SelectStar\valid\labels... 112 images, 0 backg

In [38]:
# ---------------------------------------------------------
# [단계 6-2] Step 2: 승자 모델로 1280 해상도 학습 (본 실험)
# ---------------------------------------------------------
# PC를 재시작한 후에도 이 셀만 실행하면 Step 1 결과를 불러와서 진행합니다.

import json  # PC 재시작 후 이 셀만 실행할 수 있으므로 필요

high_sz = 1280 # 본 실험 해상도

# Step 1 결과 파일 읽기
results_file = ROOT_DIR + "\\step1_comparison_results.json"

if not os.path.exists(results_file):
    print("!! Step 1 결과 파일이 없습니다!")
    print(f"   먼저 위의 Step 1 셀을 실행하여 640 해상도 비교를 완료하세요.")
else:
    with open(results_file, 'r') as f:
        comparison_results = json.load(f)
    
    if not comparison_results:
        print("!! Step 1 결과가 비어있습니다. Step 1을 다시 실행하세요.")
    else:
        # 승자 모델 결정
        winner_tag = max(comparison_results, key=comparison_results.get)
        winner_model_pt = f"{winner_tag}.pt"
        
        print("="*70)
        print(f">> Step 1 결과: 승자 모델은 {winner_tag} (mAP50: {comparison_results[winner_tag]:.4f})")
        print("="*70)
        print(f"\n>> [Step 2] {winner_tag} 모델로 1280 해상도 본 실험을 시작합니다.")
        
        run_name_high = f"Step2_Winner_{winner_tag}_SZ{high_sz}_E{FIXED_EPOCHS}"
        
        try:
            model_high = YOLO(winner_model_pt)
            model_high.train(
                data = ROOT_DIR + "\\fittogether.yaml",
                epochs = FIXED_EPOCHS,
                imgsz = high_sz,        # 입력 이미지 크기 (해상도 차이 비교)
                batch = -1,             # AutoBatch가 이미지 사이즈(1280)에 맞춰 자동 조정
                patience = 10,           # 본 실험도 조기 종료 적용
                device = 0,             # 첫 번째 GPU 사용
                project = ROOT_DIR + "\\runs",  # 결과 저장 경로
                name = run_name_high,   # 실험 결과 폴더 이름
                exist_ok = True,        # 기존 폴더가 있어도 덮어쓰기 허용
                verbose = False         # 학습 과정의 너무 긴 텍스트 출력을 줄이고 싶을 때 사용
            )
            
            print("\n" + "="*70)
            print(f">> Step 2 완료! 모든 최적화 실험이 완료되었습니다.")
            print("="*70)
            
        except Exception as e:
            print(f"\n!! Step 2 학습 중 오류 발생: {str(e)}")


>> Step 1 결과: 승자 모델은 yolov8s (mAP50: 0.5152)

>> [Step 2] yolov8s 모델로 1280 해상도 본 실험을 시작합니다.
Ultralytics 8.3.249  Python-3.12.10 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\yssong\SelectStar\fittogether.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=Step2_Winner_yol

In [48]:
# ---------------------------------------------------------
# [단계 7] 결과 수집 및 종합 비교 분석
# ---------------------------------------------------------
# Step 1, Step 2 실행 결과를 분석합니다.

import json  # 이 셀을 독립적으로 실행할 수 있으므로 필요

results_summary = []
plt.figure(figsize=(14, 8))

# Step 1 결과 로드
results_file = ROOT_DIR + "\\step1_comparison_results.json"
if os.path.exists(results_file):
    with open(results_file, 'r') as f:
        step1_results = json.load(f)
else:
    step1_results = {}

# 분석 대상 폴더 찾기
analysis_folders = []

# Step 1: 640 해상도 결과
for m_tag in ['yolov8n', 'yolov8s']:
    folder = f"Step1_{m_tag}_SZ640_E{FIXED_EPOCHS}"
    if os.path.exists(f"{ROOT_DIR}\\runs\\{folder}\\results.csv"):
        analysis_folders.append((m_tag, 640, folder))

# Step 2: 1280 해상도 결과 (승자 모델만)
if step1_results:
    winner_tag = max(step1_results, key=step1_results.get)
    folder = f"Step2_Winner_{winner_tag}_SZ1280_E{FIXED_EPOCHS}"
    if os.path.exists(f"{ROOT_DIR}\\runs\\{folder}\\results.csv"):
        analysis_folders.append((winner_tag, 1280, folder))

if not analysis_folders:
    print("!! 분석할 결과 파일이 없습니다. 먼저 Step 1과 Step 2를 실행하세요.")
else:
    print(f">> {len(analysis_folders)}개 실험 결과 분석 중...")
    
    for m_tag, sz, folder in analysis_folders:
        csv_path = f"{ROOT_DIR}\\runs\\{folder}\\results.csv"
        
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            df.columns = df.columns.str.strip()
            
            # 정확도(mAP50) 추이 시각화
            # yolov8s_SZ1280은 빨간색 점선, 나머지는 기본 설정
            if m_tag == 'yolov8s' and sz == 1280:
                ls = '--'           # 점선
                lw = 2              # 두께
                color = 'red'       # 빨간색
                marker = 'o'        # 원형 마커
            else:
                ls = '-' if sz == 640 else '--'
                lw = 2 if sz == 1280 else 1
                color = None        # 기본 색상
                marker = 'o'        # 모든 선에 원형 마커
            
            plt.plot(df['epoch'], df['metrics/mAP50(B)'], 
                    linestyle=ls, linewidth=lw, color=color, 
                    marker=marker, markersize=4, 
                    label=f"{m_tag}_SZ{sz}")
            
            # 최종 정확도를 리스트에 저장
            final_map = df['metrics/mAP50(B)'].iloc[-1]
            results_summary.append({
                'Model': m_tag, 
                'Size': sz,
                'Run': folder, 
                'mAP50': final_map,
                'Epochs': len(df)
            })

    # mAP50 = 0.5 기준선 추가 (굵은 검정색 실선)
    plt.axhline(y=0.5, color='black', linestyle='-', linewidth=1, label='Target mAP50=0.5')
    
    plt.title(f'Performance Analysis (Max {FIXED_EPOCHS} epochs with Early Stopping)', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('mAP50', fontsize=12)
    
    # 범례 순서 제어 (방법 1)
    handles, labels = plt.gca().get_legend_handles_labels()
    order = [2, 1, 0, 3] # 원하는 순서로 인덱스 지정
    plt.legend([handles[i] for i in order], [labels[i] for i in order], fontsize=10)
    
    plt.grid(True, alpha=0.3)
    plt.savefig(ROOT_DIR + f"\\Step_Comparison_E{FIXED_EPOCHS}.png", dpi=100)
    plt.show()
    
    # 결과 요약 출력
    if results_summary:
        print("\n[최종 결과 요약]")
        df_summary = pd.DataFrame(results_summary)
        print(df_summary.to_string(index=False))
        df_summary.to_csv(ROOT_DIR + "\\Final_Comparison_Report.csv", index=False)
        print(f"\n>> 결과가 저장되었습니다: Final_Comparison_Report.csv")


>> 3개 실험 결과 분석 중...


<Figure size 1400x800 with 1 Axes>


[최종 결과 요약]
  Model  Size                             Run   mAP50  Epochs
yolov8n   640         Step1_yolov8n_SZ640_E50 0.46343      50
yolov8s   640         Step1_yolov8s_SZ640_E50 0.51525      50
yolov8s  1280 Step2_Winner_yolov8s_SZ1280_E50 0.62285      50

>> 결과가 저장되었습니다: Final_Comparison_Report.csv


In [52]:
# ---------------------------------------------------------
# [단계 8] 대표 결과 이미지 시각화
# ---------------------------------------------------------
# Step 1 (640)과 Step 2 (1280) 결과를 시각적으로 비교합니다.

import json

# Step 1 결과에서 승자 확인
results_file = ROOT_DIR + "\\step1_comparison_results.json"
if not os.path.exists(results_file):
    print("!! Step 1 결과가 없습니다. 먼저 Step 1을 실행하세요.")
else:
    with open(results_file, 'r') as f:
        step1_results = json.load(f)
    
    if not step1_results:
        print("!! Step 1 결과가 비어있습니다.")
    else:
        winner_tag = max(step1_results, key=step1_results.get)
        
        # 시각화할 이미지 경로 수집
        img_paths = []
        titles = []
        
        # Step 1: 640 결과
        folder_640 = f"Step1_{winner_tag}_SZ640_E{FIXED_EPOCHS}"
        img_path_640 = f"{ROOT_DIR}\\runs\\{folder_640}\\val_batch0_pred.jpg"
        if os.path.exists(img_path_640):
            img_paths.append(img_path_640)
            titles.append(f"{winner_tag} @ 640 (Step 1)")
        
        # Step 2: 1280 결과
        folder_1280 = f"Step2_Winner_{winner_tag}_SZ1280_E{FIXED_EPOCHS}"
        img_path_1280 = f"{ROOT_DIR}\\runs\\{folder_1280}\\val_batch0_pred.jpg"
        if os.path.exists(img_path_1280):
            img_paths.append(img_path_1280)
            titles.append(f"{winner_tag} @ 1280 (Step 2)")
        
        # 이미지 시각화 (세로 배치, 간격 조절)
        if not img_paths:
            print("!! 시각화할 예측 결과 이미지가 없습니다.")
        else:
            # 세로로 배치, 제목이 들어갈 공간 확보
            fig, axes = plt.subplots(len(img_paths), 1, figsize=(12, 6*len(img_paths)), 
                                    gridspec_kw={'hspace': 0.2})  # 이미지 간 간격 (제목 공간 확보)
            if len(img_paths) == 1:
                axes = [axes]
            
            for idx, (img_path, title) in enumerate(zip(img_paths, titles)):
                img = cv2.imread(img_path)
                axes[idx].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                axes[idx].set_title(title, fontsize=20, fontweight='bold', pad=10)
                axes[idx].axis('off')
            
            plt.tight_layout(pad=1.0)  # 전체 여백
            plt.savefig(ROOT_DIR + "\\Resolution_Comparison_Result.png", dpi=100, bbox_inches='tight')
            plt.show()
            
            print(f"\n!! 시각화 완료: {len(img_paths)}개 결과 비교")
            print(f"   저장 위치: Resolution_Comparison_Result.png")


C:\Users\sjowu\AppData\Local\Temp\ipykernel_21824\2415513800.py:55: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(pad=1.0)  # 전체 여백


<Figure size 1200x1200 with 2 Axes>


!! 시각화 완료: 2개 결과 비교
   저장 위치: Resolution_Comparison_Result.png
